#### Insertion Rules  
if maximum is surpassed:
1. Give to left sibling
2. Give to right sibling
3. Split

In [ ]:
class InternalNode(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.items = []
        self.children = []
        self.min = None
        self.lsibling = None
        self.rsibling = None
        
    def update_items_leaf(self, i):
        # update internal node items[i] to be minimum of proper leaf node
        leaf = self.children[i+1]
        self.items[i] = leaf.items[0]
        
    def update_minimum_leaf(self):
        leaf = self.children[0]
        self.min = leaf.items[0]
        
    def try_give_left(self, child_idx):
        if leaf.lsibling and leaf.lsibling.has_space():
            leaf.give_left()
            if child_idx == 0: # gave left sibling internal node the tuple
                self.lsibling.update_items_leaf(len(self.lsibling.items)-1)
                self.update_minimum_leaf() # could speed up by just using leaf.items[0]
            else:
                self.update_items_leaf(child_idx - 1)
            return True
        return False
    
    def try_give_right(self, child_idx):
        if leaf.rsibling and leaf.rsibling.has_space():
            leaf.give_right()
            if child_idx == self.capacity: # gave right sibling internal node the tuple
                self.rsibling.update_minimum_leaf()
            else:
                self.update_items_leaf(child_idx)
                if child_idx == 0:
                    self.update_minimum_leaf()
            return True
        return False
        
    def rotate(self, child_idx):
        if try_give_left(child_idx):
            return
        elif try_give_right(child_idx):
            return
        else: # Split
            leaf.split()
        
    def insert_into_leaf(self, tup):
        # find leaf and index to update
        child_idx = len(self.items)
        for i, item in enumerate(self.items):
            if tup < item:
                child_idx = i
        leaf = self.children[child_idx]
        
        # insert tuple into leaf node
        leaf.insert(tup)
        
        # fix leaf node if it is over capacity
        if leaf.over_capacity():
            self.rotate(child_idx)

In [ ]:
class LeafNode(object):
    def __init__(self, capacity, items = []):
        self.capacity = capacity
        self.items = items
        self.lsibling = None
        self.rsibling = None

    def __repr__(self):
        s = ''
        s += 'Leaf Node\n'
        s += f'Capacity: {self.capacity}\n'
        s += f'Items: {self.items}\n'
        s += f'children: {self.children}\n'
        return s
    
    def has_space(self):
        return len(self.items) < self.capacity
    
    def over_capacity(self):
        return len(self.items) > self.capacity
    
    def insert(self, tup):
        self.items.append(tup)
        sorted(self.items)
    
    def give_left(self):
        item = self.items[0]
        del self.items[0]
        self.lsibling.insert(item)
    
    def give_right(self):
        item = self.items[-1]
        del self.items[-1]
        self.rsibling.insert(item)
        
    def split(self):
        # split items
        self.items = self.items[:self.c//2]
        new_right = LeafNode(self.c, self.items[self.c//2:])
        old_right = self.rsibling
        
        # change siblings
        self.rsibling = new_right
        new_right.lsibling = self
        new_right.rsibling = old_right
        old_right.lsibling = new_right

In [57]:
m = 3 # elements in internal nodes
l = 4 # elements in leaf nodes

In [58]:
ln1 = LeafNode(l)

for i in range(5):
    ln1.insert(i)

print(ln1)

Empty items - insert 0
Larger than all items - inserting 1 at the end
Larger than all items - inserting 2 at the end
Larger than all items - inserting 3 at the end
Larger than all items - inserting 4 at the end
Leaf Node
Capacity: 4
Items: [0, 1, 2, 3, 4]
children: []

